# SYNTHALIS DOCKING
##  Upgraded Synthalis Molecular Docking Workflow

## Imports and Setup

In [1]:
import os
import sys
import re
import shutil
import requests
import numpy as np
import pandas as pd
import py3Dmol
import subprocess
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display, clear_output, HTML
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')  # Suppress RDKit warnings

## Configuration

## Check Tools

In [2]:
SYNTHALISDOCK_PATH = Path("/home/imraan/Downloads/Jupyter_Dock")  # UPDATE THIS PATH
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "synthalisdock_data"
RECEPTOR_DIR = DATA_DIR / "receptors"
LIGAND_DIR = DATA_DIR / "ligands"
RESULTS_DIR = DATA_DIR / "results"

# Create directories
for d in [DATA_DIR, RECEPTOR_DIR, LIGAND_DIR, RESULTS_DIR]:
    d.mkdir(exist_ok=True, parents=True)

print("✅ Environment initialized")

✅ Environment initialized


In [3]:
def verify_installation():
    print("🔍 Checking minimal requirements:")
    
    # Check Vina
    try:
        vina_path = shutil.which("vina")
        assert vina_path
        print(f"✅ AutoDock Vina: {vina_path}")
    except:
        print("❌ AutoDock Vina not found (required)")
    
    # Check RDKit
    try:
        import rdkit
        print(f"✅ RDKit: {rdkit.__version__}")
    except:
        print("❌ RDKit not installed (required)")
    
    # Check OpenBabel (optional)
    try:
        obabel_path = shutil.which("obabel")
        if obabel_path:
            print(f"⚠ OpenBabel found but not required: {obabel_path}")
    except:
        pass

verify_installation()

🔍 Checking minimal requirements:
✅ AutoDock Vina: /home/imraan/miniforge3/envs/molecule_dock/bin/vina
✅ RDKit: 2024.03.6
⚠ OpenBabel found but not required: /home/imraan/Downloads/Jupyter_Dock/mgltools_x86_64Linux2_1.5.7/bin/obabel


## Ligand Prep

In [4]:
def prepare_ligand(smiles, idx):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if not mol:
            raise ValueError("Invalid SMILES")
        
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol)
        AllChem.MMFFOptimizeMolecule(mol)
        
        pdbqt_file = LIGAND_DIR / f"ligand_{idx}.pdbqt"
        with open(pdbqt_file, 'w') as f:
            f.write(Chem.MolToPDBBlock(mol))
        
        return pdbqt_file
    except Exception as e:
        print(f"❌ Ligand {idx} failed: {e}")
        return None
# Test
test_lig = prepare_ligand("CN1C=NC2=C1C(=O)N(C(=O)N2C)C", 1)
print(f"Ligand prepared at: {test_lig}")

Ligand prepared at: /home/imraan/Jupyter_Dock/synthalisdock_data/ligands/ligand_1.pdbqt


## Receptor Prep

In [5]:
def prepare_receptor(uniprot_id):
    """Download and prepare receptor without PDBQT conversion"""
    try:
        pdb_file = RECEPTOR_DIR / f"receptor_{uniprot_id}.pdb"
        
        # Download from AlphaFold
        if not pdb_file.exists():
            url = f"https://alphafold.ebi.ac.uk/files/AF-{uniprot_id}-F1-model_v4.pdb"
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            with open(pdb_file, 'w') as f:
                f.write(response.text)
        
        # Create minimal PDBQT file (without charges)
        pdbqt_file = RECEPTOR_DIR / f"receptor_{uniprot_id}.pdbqt"
        with open(pdb_file) as fin, open(pdbqt_file, 'w') as fout:
            for line in fin:
                if line.startswith(("ATOM", "HETATM")):
                    # Add zero charges and radii
                    fout.write(line[:56] + "  0.00  0.00\n")
        
        return pdbqt_file
    except Exception as e:
        print(f"❌ Failed to prepare receptor: {e}")
        return None

# Test
test_rec = prepare_receptor("E1JJ46")
print(f"Receptor prepared at: {test_rec}")

Receptor prepared at: /home/imraan/Jupyter_Dock/synthalisdock_data/receptors/receptor_E1JJ46.pdbqt


## Docking

In [6]:
def run_docking(receptor_pdbqt, ligand_pdbqt, output_dir):
    try:
        output_dir.mkdir(exist_ok=True, parents=True)
        
        result = subprocess.run([
            "vina",
            "--receptor", str(receptor_pdbqt),
            "--ligand", str(ligand_pdbqt),
            "--center_x", "15", "--center_y", "15", "--center_z", "15",
            "--size_x", "30", "--size_y", "30", "--size_z", "30",
            "--exhaustiveness", "8",
            "--out", str(output_dir / "docked.pdbqt"),
            "--log", str(output_dir / "log.txt")
        ], capture_output=True, text=True, timeout=300)
        
        # Parse scores
        scores = []
        if (output_dir / "log.txt").exists():
            with open(output_dir / "log.txt") as f:
                for line in f:
                    if "Affinity" in line:
                        try:
                            scores.append(float(line.split()[1]))
                        except:
                            continue
        return scores
    except Exception as e:
        print(f"❌ Docking failed: {e}")
        return None

## Input

In [7]:
default_uniprots = ["P00520", "", "", "", ""]  # First one is tyrosine kinase
default_smiles = [
    "CN1C=NC2=C1C(=O)N(C(=O)N2C)C",  # Caffeine
    "C1=CC=C(C=C1)C=O",               # Benzaldehyde
    "CC(=O)OC1=CC=CC=C1C(=O)O",       # Aspirin
    "C1CCCCC1",                       # Cyclohexane
    "CCO"                             # Ethanol
]

# Create input boxes
uniprot_boxes = [
    widgets.Text(
        value=default_uniprots[i],
        placeholder=f'UniProt ID {i+1}',
        layout=widgets.Layout(width='90%')
    ) for i in range(5)
]

smiles_boxes = [
    widgets.Text(
        value=default_smiles[i],
        placeholder=f'SMILES {i+1}',
        layout=widgets.Layout(width='90%')
    ) for i in range(5)
]

# Create labels
uniprot_labels = [widgets.Label(f"Receptor {i+1}") for i in range(5)]
smiles_labels = [widgets.Label(f"Ligand {i+1}") for i in range(5)]

# Create button
process_btn = widgets.Button(
    description="Run Docking",
    button_style='success',
    layout=widgets.Layout(width='200px', margin='20px 0')
)

# Create output area
output = widgets.Output()

# Display UI
display(widgets.VBox([
    widgets.HTML("<h2 style='color:#1a5276; margin-bottom:20px'>Synthalis Docking</h2>"),
    
    widgets.HTML("<h3 style='color:#2874a6; margin-bottom:10px'>Receptors (UniProt IDs)</h3>"),
    widgets.VBox([widgets.HBox([uniprot_labels[i], uniprot_boxes[i]]) for i in range(5)]),
    
    widgets.HTML("<h3 style='color:#2874a6; margin-top:20px; margin-bottom:10px'>Ligands (SMILES)</h3>"),
    widgets.VBox([widgets.HBox([smiles_labels[i], smiles_boxes[i]]) for i in range(5)]),
    
    process_btn,
    output
]))

## Visualise

In [8]:
def visualize_results(receptor_pdb, ligand_pdb, docked_pdbqt=None):
    viewer = py3Dmol.view(width=800, height=600)
    
    # Receptor
    with open(receptor_pdb) as f:
        viewer.addModel(f.read(), 'pdb')
    viewer.setStyle({'cartoon': {'color': 'spectrum'}})
    
    # Original ligand
    with open(ligand_pdb) as f:
        viewer.addModel(f.read(), 'pdb')
    viewer.setStyle({'model': -1}, {'stick': {'colorscheme': 'greenCarbon'}})
    
    # Docked pose
    if docked_pdbqt and os.path.exists(docked_pdbqt):
        with open(docked_pdbqt) as f:
            viewer.addModel(f.read(), 'pdbqt')
        viewer.setStyle({'model': -1}, {'stick': {'colorscheme': 'redCarbon'}})
    
    viewer.zoomTo()
    return viewer

## Main Workflow

In [9]:
def on_run_clicked(btn):
    with output_area:
        output_area.clear_output()
        print("⚡ Starting Docking Workflow")
        
        try:
            # Get inputs
            uniprot_ids = [b.value.strip() for b in uniprot_inputs if b.value.strip()]
            smiles_list = [b.value.strip() for b in smiles_inputs if b.value.strip()]
            
            if not uniprot_ids or not smiles_list:
                raise ValueError("Please provide at least 1 receptor and 1 ligand")
            
            print(f"\n🔬 Input Summary:")
            print(f"- Receptors: {', '.join(uniprot_ids)}")
            print(f"- Ligands: {len(smiles_list)} provided")
            
            # Prepare receptors
            print("\n🛠️ Preparing Receptors...")
            rec_files = []
            for uid in uniprot_ids:
                print(f"  - Processing {uid}...")
                rec_file = prepare_receptor(uid)
                if rec_file:
                    rec_files.append(rec_file)
                    print(f"    ✅ Prepared {rec_file.name}")
                else:
                    print(f"    ❌ Failed to prepare {uid}")
            
            # Prepare ligands
            print("\n🧪 Preparing Ligands...")
            lig_files = []
            for i, smi in enumerate(smiles_list, 1):
                print(f"  - Processing ligand {i}...")
                lig_file = prepare_ligand(smi, i)
                if lig_file:
                    lig_files.append(lig_file)
                    print(f"    ✅ Prepared {lig_file.name}")
                else:
                    print(f"    ❌ Failed to prepare ligand {i}")
            
            if not rec_files or not lig_files:
                raise ValueError("Structure preparation failed")
            
            # Run docking
            print("\n⚡ Running Docking...")
            results = []
            
            for rec_file in rec_files:
                for lig_file in lig_files:
                    rec_name = Path(rec_file).stem.replace("receptor_", "")
                    lig_name = Path(lig_file).stem.replace("ligand_", "")
                    dock_dir = RESULTS_DIR / f"{rec_name}_vs_{lig_name}"
                    
                    print(f"\n🔬 Docking {rec_name} + {lig_name}")
                    scores = run_docking(rec_file, lig_file, dock_dir)
                    
                    if scores:
                        best_score = min(scores)
                        avg_score = np.mean(scores)
                        results.append({
                            'Receptor': rec_name,
                            'Ligand': lig_name,
                            'Best Score': f"{best_score:.2f}",
                            'Average': f"{avg_score:.2f}",
                            'Directory': str(dock_dir)
                        })
                        print(f"    ✅ Success! Best score: {best_score:.2f} kcal/mol")
                    else:
                        print("    ❌ Docking failed")
            
            # Show results
            if results:
                print("\n📊 Docking Results:")
                display(pd.DataFrame(results))
                
                # Visualize first result
                first = results[0]
                rec_pdb = RECEPTOR_DIR / f"receptor_{first['Receptor']}.pdb"
                lig_pdb = LIGAND_DIR / f"ligand_{first['Ligand'].split('_')[0]}.pdb"
                docked_pdbqt = Path(first['Directory']) / "docked.pdbqt"
                
                print("\n👀 Visualizing first result:")
                viewer = py3Dmol.view(width=800, height=600)
                with open(rec_pdb) as f:
                    viewer.addModel(f.read(), 'pdb')
                viewer.setStyle({'cartoon': {'color': 'spectrum'}})
                
                with open(lig_pdb) as f:
                    viewer.addModel(f.read(), 'pdb')
                viewer.setStyle({'model': -1}, {'stick': {'colorscheme': 'greenCarbon'}})
                
                if docked_pdbqt.exists():
                    with open(docked_pdbqt) as f:
                        viewer.addModel(f.read(), 'pdbqt')
                    viewer.setStyle({'model': -1}, {'stick': {'colorscheme': 'redCarbon'}})
                
                viewer.zoomTo()
                display(viewer)
            else:
                print("\n❌ No successful docking results")
            
            print("\n🎉 Workflow completed!")
        
        except Exception as e:
            print(f"\n❌ Error: {str(e)}")

# Connect button
run_btn.on_click(on_run_clicked)
print("✅ Ready to dock! Enter proteins/ligands and click the button")

NameError: name 'run_btn' is not defined